In [11]:
import csv
import re
import os
import random
from datetime import datetime
from datetime import timedelta
from glob import glob

# Data pre-processing
## Define the input and output folders

In [12]:
PREDICTION_HORIZON = 20
inputFolder = 'tblADataRTCGM_Unblinded_ControlGroup_1'
patients = glob(inputFolder + '/*')
outputFolder = inputFolder + '_output_' + str(PREDICTION_HORIZON)
if not os.path.exists(outputFolder):
    os.makedirs(outputFolder)

In [13]:
patients

['tblADataRTCGM_Unblinded_ControlGroup_1/155.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/140.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/142.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/180.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/147.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/146.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/178.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/150.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/145.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/151.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/136.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/122.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/123.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/121.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/135.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/109.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/108.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/118.csv',
 'tblADataRTCGM_Unblinded_ControlGroup_1/18.csv',
 'tblADataRTCGM_Unblinded_Contro

## The data strategy
Goal is to create data points consisting of a blood sugar measurement at a
 prediction time horizon of 30 mins, and measurements at the following times:
   - Current time
   - Current time - 10 mins
   - Current time - 20 mins
   - Current time - 30 mins
   - Current time - 40 mins
   - Current time - 60 mins
   - Current time - 90 mins

 When the specified measurement time does not coincide with a measurement,
 the blood sugar level at the specified time will be determined by linear
 interpolation of the two nearest measurement points.

 Data points that represent potentially dangerous sugar lows or highs will be
 flagged for use in validation.

 Measurement points will only be used if they meet the following criteria:
   - Values determined from linear interpolation must be determined from
       measurements separated by no more than 11 mins
   - All values must be taken from between the hours of 11pm and 7am. In other
       words, the first prediction time considered will be ~1am.


 Performs a linear interpolation of the blood sugar measurements between two
 input data points.

In [14]:
def interpolate(prev, cur, desiredTime) :
    totalTime = (cur[0] - prev[0]).total_seconds()
    prevToDesiredTime = (desiredTime - prev[0]).total_seconds()
    # Perform linear interpolation:
    sugar = prevPoint[1] + (((curPoint[1] - prevPoint[1]) / totalTime) * prevToDesiredTime)
    return sugar

In [15]:
for patientFolder in patients :
    patientID = re.split("/", patientFolder)[1][:-4]

In [16]:
os.getcwd()

'/Users/blaze/OneDrive/Work/HealthCare/Project 9 -- Deep Learning RNN for Predicting  Glucose Concentration for Diabetes1 Patients'

In [17]:
re.split("/", patientFolder)[1]

'170.csv'

In [103]:
open('tblADataRTCGM_Unblinded_ControlGroup_1/155.csv', newline='')

TypeError: 'newline' is an invalid keyword argument for this function

[]

## Generate the training and testing data

In [19]:
# Read input file
for patientFolder in patients :
    patientID = re.split("/", patientFolder)[1][:-4]
    outputTrainFile = outputFolder + '/' + patientID + '_train.csv'
    outputTestFile = outputFolder + '/' + patientID + '_test.csv'
    open(outputTrainFile, 'w').close()
    open(outputTestFile, 'w').close()
    days = glob(patientFolder + '/*')
    for dayFile in days :
        # Create new empty lists
        data = []
        processedData_Train = []
        processedData_Test = []
        with open(dayFile, newline = '') as f:
            reader = csv.reader(f)
            # Populate the list with a full day worth of data
            # (from 12 noon until 12 noon to ensure that the complete night is included)
            for row in reader :
                #curTime = dateutil.parser.parse(row[2]) # Parse date
                #curTime = datetime.strptime(row[2], '%Y-%m-%d %H:%M:%S.%f') # Example date: "2001-03-03 12:01:00.000000"
                #bloodSugar = float(row[3]) # Parse blood sugar
                curTime = datetime.strptime(row[0], '%m\%d\%Y %H:%M')
                bloodSugar = float(row[1])
                newTuple = (curTime, bloodSugar)
                data.append(newTuple)
        # End with file

        # Make sure data is sorted by time:
        data.sort(key=lambda tup: tup[0])

        # For each data point:
        for index, predictionPoint in enumerate(data) :
            valid = 1
            # Only consider points between 1AM and 7AM:
            if predictionPoint[0].hour >= 1 and predictionPoint[0].hour < 7 :
                # Determine all desired measurement times:
                time1 = predictionPoint[0] - timedelta(minutes = PREDICTION_HORIZON) # 30 mins ago
                tenMinutes = timedelta(minutes = 10)
                time2 = time1 - tenMinutes # 40 mins ago
                time3 = time2 - tenMinutes # 50 mins ago
                time4 = time3 - tenMinutes # 60 mins ago
                time5 = time4 - tenMinutes # 70 mins ago
                time6 = time5 - timedelta(minutes = 20) # 90 mins ago
                time7 = time6 - timedelta(minutes = 30) # 120 mins ago

                # Initialize all blood sugar measurements
                bloodSugar1 = -1.0
                bloodSugar2 = -1.0
                bloodSugar3 = -1.0
                bloodSugar4 = -1.0
                bloodSugar5 = -1.0
                bloodSugar6 = -1.0
                bloodSugar7 = -1.0

                curPoint = predictionPoint
                # Iterate from current index backwards
                for i in range(index - 1, -1, -1) :
                    prevPoint = data[i]
                    # If prevPoint and curPoint aren't too far apart
                    if (prevPoint[0] + timedelta(minutes = 12)) > curPoint[0] :
                        if prevPoint[0] <= time1 and curPoint[0] > time1 : # Straddling time1
                            bloodSugar1 = interpolate(prevPoint, curPoint, time1)
                        elif prevPoint[0] <= time2 and curPoint[0] > time2 : # Straddling time2
                            bloodSugar2 = interpolate(prevPoint, curPoint, time2)
                        elif prevPoint[0] <= time3 and curPoint[0] > time3 : # Straddling time3
                            bloodSugar3 = interpolate(prevPoint, curPoint, time3)
                        elif prevPoint[0] <= time4 and curPoint[0] > time4 : # Straddling time4
                            bloodSugar4 = interpolate(prevPoint, curPoint, time4)
                        elif prevPoint[0] <= time5 and curPoint[0] > time5 : # Straddling time5
                            bloodSugar5 = interpolate(prevPoint, curPoint, time5)
                        elif prevPoint[0] <= time6 and curPoint[0] > time6 : # Straddling time6
                            bloodSugar6 = interpolate(prevPoint, curPoint, time6)
                        elif prevPoint[0] <= time7 and curPoint[0] > time7 : # Straddling time7
                            bloodSugar7 = interpolate(prevPoint, curPoint, time7)
                    curPoint = prevPoint # Update curPoint for next iteration

                # If all bloodSugar measuremets were determined:
                if  (bloodSugar1 > 0) and (bloodSugar2 > 0) and (bloodSugar3 > 0) and (bloodSugar4 > 0) and (bloodSugar5 > 0) and (bloodSugar6 > 0) and (bloodSugar7 > 0) :
                    newPoint = (bloodSugar7, bloodSugar6, bloodSugar5, bloodSugar4, bloodSugar3, bloodSugar2, bloodSugar1, predictionPoint[1])
                    randNum = random.random() #Random float between 0 and 1
                    if randNum > 0.25 :
                        processedData_Train.append(newPoint) # Added to training set with probability of 75%
                    else :
                        processedData_Test.append(newPoint)
            # End if predictionPoint is between 1AM and 7AM
        # End for each input data predictionPoint

        # Write results to output files:
        # Ouput file contains blood sugar measurments with following format on each line:
        # '90 mins ago','60 mins ago','40 mins ago','30 mins ago','20 mins ago','10 mins ago','cur time','30 mins ahead'
        with open(outputTrainFile, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(processedData_Train)
            print(processedData_Train)

        with open(outputTestFile, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(processedData_Test)



# Building the Feed Froward Neural Network Model

 The network has 7 input nodes and 1 output node. If the current time is 'T',
 then the inputs and output represent the blood glucose measurements at the
 following times:
   Inputs:     - T
               - (T - 10 mins)
               - (T - 20 mins)
               - (T - 30 mins)
               - (T - 40 mins)
               - (T - 50 mins)
               - (T - 60 mins)

   Output:     - (T + 20 mins)

In [57]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import matplotlib.pyplot as plt
NUM_EPOCHS = 1500 # Number of training epochs

In [58]:
# readData reads data from the specified pre-processed input data file.
# The function returns an array of input data points and an array of the
# corresponding desired outputs.
def readData(filePath) :
    x_data = []
    y_data = []
    with open(filePath, 'r') as f:
        for line in f:
            values = line.split(',')
            time1 = float(values[0])
            time2 = float(values[1])
            time3 = float(values[2])
            time4 = float(values[3])
            time5 = float(values[4])
            time6 = float(values[5])
            time7 = float(values[6])
            time8 = float(values[7])
            newPointx = [time1, time2, time3, time4, time5, time6, time7] # Input
            newPointy = [time8] # Desired Output
            x_data.append(newPointx)
            y_data.append(newPointy)
    data = [x_data, y_data]
    return data;

## define the evaluation metrics and the coresponding generating function
 evaluateNetwork runs the trained network on the the provided network and
 reports the following evaluation metrics:
   - mean squared prediction error
   - percentage of lows that were correctly identified
   - percentage of highs that were corretly identified
   - number of falsely reported lows
   - number of falsely reported highs

 These metrics are defined as follows:
   - MSE:
       -> Average of (y_desired - y_actual)^2 for each test point
   - Low prediction accuracy:
       -> 100 * (Number of correct lows) / (Number of lows)
       -> Lows are any blood glucose level less than 70 mg/dL
   - High prediction accuracy:
       -> 100 * (Number of correct highs) / (Number of highs)
       -> Highs are any blood glucose level greater than 200
   - Number of false lows:
       -> Number of false lows where (y_desired - y_actual) > 6
       -> Note: false alarms are not counted if the prediction error is small
   - Number of false highs:
       -> Number of false highs where (y_actual - y_desired) > 6
       -> Note: false alarms are not counted if the prediciton error is small

In [59]:
def evaluateNetwork(session, inData, outData, prediction) :
    # Compute mse:
    mse = session.run(tf.reduce_mean(tf.square(prediction - y_desired)), feed_dict={x: inData, y_desired: outData})
    numTestPoints = len(inData)
    numPredictedLows = 0
    numLows = 0
    numFalseLows = 0
    numPredictedHighs = 0
    numHighs = 0
    numFalseHighs = 0
    for i, inputPoint in enumerate(inData) :
        # Apply network on current point:
        predicted = session.run(prediction, feed_dict={x: [inputPoint]})
        desired = outData[i][0]


        # Update numLows, numHighs:
        if(desired < 70) :
            numLows += 1
        elif(desired > 200) :
            numHighs += 1

        # Update prediction counts:
        if(predicted < 70) : # If predicted low
            if(desired < 70) : # If low prediction was correct
                numPredictedLows += 1
            elif((desired - predicted) > 8) : # If low prediction was incorrect and error was 'large'
                numFalseLows += 1
        elif(predicted > 200) : # If predicted high
            if(desired > 200) : # If high prediction was correct
                numPredictedHighs += 1
            elif((predicted - desired) > 8) : # If high prediction was incorrect and error was 'large'
                numFalseHighs += 1
    # Print results:
    print('Number of test points: ', numTestPoints)
    print('Number of lows: ', numLows)
    print('Number of highs: ', numHighs)
    print("Number of 'normal' points: ", numTestPoints - numLows - numHighs)
    print('') # New line
    print('MSE: ', mse)
    print('')
    print('Low prediction accuracy: ', 100 * numPredictedLows / numLows, '%')
    print('Number of false lows: ', numFalseLows)
    print('')
    print('High prediction accuracy: ', 100 * numPredictedHighs / numHighs, '%')
    print('Number of false highs: ', numFalseHighs)
# End evaluateNetwork(...)

In [60]:
x = tf.placeholder(tf.float32, [None, 7], name='x') # Input placeholder
y_desired = tf.placeholder(tf.float32, [None, 1], name='y_desired') # Desired output placeholder

In [61]:
x

<tf.Tensor 'x_2:0' shape=(?, 7) dtype=float32>

In [62]:
def feedForwardNN(x) :
    # Define the weights from inputs to first hidden layer (15 nodes):
    Wh1 = tf.Variable(tf.random_uniform([7, 15], minval = -1, maxval = 1, dtype = tf.float32))
    # Bias for first hidden layer:
    bh1 = tf.Variable(tf.zeros([1, 15]))

    # Define the weights from first hidden layer to second (15 nodes):
    Wh2 = tf.Variable(tf.random_uniform([15, 15], minval = -1, maxval = 1, dtype = tf.float32)) # The weights from each of the 784 inputs to the 10 output nodes
    # Bias for second hidden layer:
    bh2 = tf.Variable(tf.zeros([1, 15])) # One bias input for each of the 10 output nodes

    # Define the weights from second hidden layer to output layer (1 node):
    Wo = tf.Variable(tf.random_uniform([15, 1], minval = -1, maxval = 1, dtype = tf.float32))
    # Bias to output node:
    bo = tf.Variable(tf.zeros([1, 1]))

    # Generate the FFNN network
    h1 = tf.add(tf.matmul(x, Wh1), bh1) # Hidden layer 1 output
    h2 = tf.add(tf.matmul(h1, Wh2), bh2) # Hidden layer 2 output
    output = tf.add(tf.matmul(h2, Wo), bo) # Network output

    return output

## Training Proessing

In [85]:
def trainFFNN(x):
    # Import the training data and test data:
    # 151, 149, 174
    trainData_in, trainData_out = readData('tblADataRTCGM_Unblinded_ControlGroup_1_output_20/174_train.csv')
    testData_in, testData_out = readData('tblADataRTCGM_Unblinded_ControlGroup_1_output_20/174_test.csv')

    prediction = feedForwardNN(x)

    # Error function to be minimized is the mean square error:
    loss = tf.reduce_mean(tf.square(prediction - y_desired))

    # Define training algorithm (Adam Optimizer):
    # Note: AdamOptimizer produced better results than the GradientDescentOptimizer
    #train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_step = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

    # Train:
    errors = []
    sess = tf.InteractiveSession()
    #tf.global_variables_initializer
    
    init = tf.global_variables_initializer()
    for i in range(NUM_EPOCHS): # 1000 training epochs
        ### Batch training was tested, but per-epoch produced better results:
        # Train with one batch at a time:
        #for start, end in zip(range(0, len(trainData_in), BATCH_SIZE), range(BATCH_SIZE, len(trainData_in), BATCH_SIZE)):
            #sess.run(train_step, feed_dict={x: trainData_in[start:end], y_desired: trainData_out[start:end]})

        # Per-Epoch training:
        sess.run(init, feed_dict={x: trainData_in, y_desired: trainData_out})
        # Print MSE on test data after every 10 epochs
        if i % 10 == 0:
            mse = sess.run(tf.reduce_mean(tf.square(prediction - y_desired)), feed_dict={x: testData_in, y_desired: testData_out})
            errors.append(mse)
            print(mse)

    # Output the desired and actual outputs for each test data point
    #for i, inputPoint in enumerate(testData_in) :
    #    output = sess.run(y, feed_dict={x: [inputPoint]})
    #    print('desired: ', testData_out[i], ', actual: ', output)

    # Test:
    print('Patient 174 data:')
    evaluateNetwork(sess, testData_in, testData_out, prediction)
    print('Patient 149 data:')
    testData_in, testData_out = readData('tblADataRTCGM_Unblinded_ControlGroup_1_output_20/149_test.csv')
    evaluateNetwork(sess, testData_in, testData_out, prediction)
    print('Patient 151 data:')
    testData_in, testData_out = readData('tblADataRTCGM_Unblinded_ControlGroup_1_output_20/151_test.csv')
    evaluateNetwork(sess, testData_in, testData_out, prediction)
    # Uncomment this to evaluate the current network on a different patient:
    #testData_in, testData_out = readData('tblADataRTCGM_Blind_Baseline_Split_output/78_test.csv')
    #evaluateNetwork(sess, testData_in, testData_out, prediciton)

    # Plot the MSE throughout training
    #plt.plot(errors)
    #plt.xlabel('#epochs')
    #plt.ylabel('MSE')
    #plt.show()
# End trainFFNN(x)

Let's try to train and evalue the performance of the our FFNN:

In [82]:
trainFFNN(x)

3143655.2
2098803.0
163402.73
1436603.4
597857.6
1265184.8
894934.8
4017109.8
2873486.0
2341409.5
679770.0
16535.791
2819171.8
2678143.5
3277605.8
1177102.9
573267.44
202795.06
1858651.9
257663.44
72567.164
231765.58
37563.91
1423949.9
397625.88
3952893.2
370551.25
877712.0
248722.77
649146.4
597026.94
28260.71
5383620.5
3956015.5
2826423.2
690672.1
3432848.8
5066525.5
1783709.2
538851.44
289013.94
2845549.5
6499.5503
325148.38
3782.4075
43190.82
230467.16
3503061.2
249861.84
26292.014
1198333.8
956550.3
8715.05
37621.184
640085.8
1171535.8
1816937.1
160295.88
128779.27
201975.56
59716.734
683425.06
458186.34
595263.2
7590.816
240371.77
3093866.5
2741090.8
1834089.4
343981.06
935493.1
2911213.5
27815.96
205144.81
90585.95
25290.451
3035599.2
3467593.2
639696.3
3203.5386
2882804.5
419494.1
1876406.2
4910376.0
120485.27
742360.3
280217.38
1460133.5
1555506.8
1803263.5
1383842.6
1173347.6
3589.295
717553.75
9478921.0
303569.6
2187126.8
8764990.0
1036579.06
1228194.2
366389.44
4796006.0
13

In [86]:
trainFFNN(x)

FailedPreconditionError: Attempting to use uninitialized value beta2_power_16
	 [[Node: beta2_power_16/read = Identity[T=DT_FLOAT, _class=["loc:@Variable_100"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](beta2_power_16)]]

Caused by op u'beta2_power_16/read', defined at:
  File "/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/anaconda2/lib/python2.7/site-packages/tornado/ioloop.py", line 1064, in start
    handler_func(fd_obj, events)
  File "/anaconda2